## Load Packages

In [1]:
using Distributions
include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Long-Run Returns as a Sum of Short-Run Returns

Let $R_{t}$ be a one-period net return. 
It is sometimes convenient to approximate the $q$-period return $Z_{q}$ as

$\begin{align}
Z_{q}  &  =(R_{1}+1)(R_{2}+1)\ldots(R_{q}+1)-1\nonumber\\
&  \approx R_{1}+R_{2}+\ldots+R_{q} 
\end{align}$

Log returns $r=\ln (1+R)$ are better suited for multi-period analysis since there is no approximation error in

$z_{q}=r_{1}+r_{2}+\ldots+r_{q}$

In [3]:
R_A = [5;-5;5]/100            #net returns portfolio A in three periods
R_B = [20;-35;25]/100

r_A = log.(1+R_A)             #log returns
r_B = log.(1+R_B)   

Price_A = cumprod(1+R_A)     #price of portfolio A, assuming price in t=0 is 1.0
Price_B = cumprod(1+R_B)

println("time      Price_A      Price_B")
printmat([(1:3) Price_A Price_B])

println("Average net returns: A and B, in %")
printmat(mean([R_A R_B],1)*100)

println("Average log returns: A and B, in %")
printmat(mean([r_A r_B],1)*100)

time      Price_A      Price_B
     1.000     1.050     1.200
     2.000     0.997     0.780
     3.000     1.047     0.975

Average net returns: A and B, in %
     1.667     3.333

Average log returns: A and B, in %
     1.543    -0.844



# Distribution of Long-Run Returns in the iid Case

If $r$ is iid N$(\mu,\sigma^2)$, then $z_q$ is N$(q\mu,q\sigma^2)$. 

In [4]:
function ΦNS(x,μ=0,σ²=1)                          #cdf of N(μ,σ²), NS for "non-standard"
    Pr = cdf(Normal(μ,sqrt(σ²)),x)
  return Pr
end

function ϕNS(x,μ=0,σ²=1)              #pdf of N(μ,σ²), NS for "non-standard"
    pdfx = pdf(Normal(μ,sqrt(σ²)),x)
    return pdfx
end

ϕNS (generic function with 3 methods)

## Pdfs of Long-Run Returns (for Different Horizons)

In [5]:
Z = linspace(-50,100,101)
μ = 0.06*100
σ = 0.19*100
pdf_Z1y  = ϕNS.(Z,μ,σ^2)              #density functions of returns
pdf_Z10y = ϕNS.(Z,10*μ,10*σ^2)

plot(Z,pdf_Z1y,color=:red,linewidth=2,label="1y")
plot!(Z,pdf_Z10y,color=:blue,linewidth=2,label="10y")
title!("pdf(long run excess returns)")
xlabel!("excess return, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -50 
 
 
 -25 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 0.020 
 
 
 pdf(long run excess returns) 
 
 
 excess return, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,363.819 44.8976,362.61 50.4252,361.134 55.9528,359.347 61.4803,357.197 67.0079,354.629 72.5354,351.583 78.063,347.998 83.5906,343.807 89.1181,338.944 
 94.6457,333.344 100.173,326.943 105.701,319.683 111.228,311.51 116.756,302.382 122.283,292.269 127.811,281.156 133.339,269.044 138.866,255.957 144.394,241.939 
 149.921,227.06 155.449,211.414 160.976,195.12 166.504,178.326 172.031,161.202 177.559,143.939 183.087,126.75 188.614,109.863 194.142,93.5155 199.669,77.9513 
 205.197,63.4137 210.724,50.1391 216.252,38.3512 221.78,28.2541 227.307,20.0269 232.835,13.8184 238.362,9.74225 243.89,7.87402 249.417,8.24844 254.945,10.8585 
 260.472,15.6558 266,22.5517 271.528,31.4203 277.055,42.1023 282.583,54.4095 288.11,68.1307 293.638,83.0373 299.165,98.8898 304.693,115.444 310.22,132.458 
 315.748,149.696 321.276,166.935 326.803,183.97 332.331,200.615 337.858,216.707 343.386,232.11 348.913,246.712 354.441,260.426 359.969,273.192 365.496,284.972 
 371.024,295.751 376.551,305.533 382.079,314.338 387.606,322.202 393.134,329.17 398.661,335.297 404.189,340.644 409.717,345.275 415.244,349.257 420.772,352.655 
 426.299,355.535 431.827,357.957 437.354,359.98 442.882,361.658 448.409,363.04 453.937,364.17 459.465,365.088 464.992,365.828 470.52,366.421 476.047,366.893 
 481.575,367.266 487.102,367.558 492.63,367.786 498.157,367.963 503.685,368.099 509.213,368.202 514.74,368.281 520.268,368.34 525.795,368.385 531.323,368.418 
 536.85,368.442 542.378,368.46 547.906,368.473 553.433,368.483 558.961,368.489 564.488,368.494 570.016,368.498 575.543,368.5 581.071,368.502 586.598,368.503 
 592.126,368.504 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,347.157 44.8976,346.165 50.4252,345.142 55.9528,344.088 61.4803,343.001 67.0079,341.883 72.5354,340.734 78.063,339.552 83.5906,338.34 89.1181,337.096 
 94.6457,335.821 100.173,334.515 105.701,333.18 111.228,331.815 116.756,330.421 122.283,328.998 127.811,327.548 133.339,326.071 138.866,324.569 144.394,323.041 
 149.921,321.49 155.449,319.916 160.976,318.321 166.504,316.705 172.031,315.071 177.559,313.42 183.087,311.753 188.614,310.071 194.142,308.378 199.669,306.674 
 205.197,304.962 210.724,303.243 216.252,301.519 221.78,299.792 227.307,298.065 232.835,296.339 238.362,294.617 243.89,292.901 249.417,291.193 254.945,289.497 
 260.472,287.813 266,286.144 271.528,284.494 277.055,282.864 282.583,281.257 288.11,279.675 293.638,278.12 299.165,276.596 304.693,275.104 310.22,273.647 
 315.748,272.227 321.276,270.847 326.803,269.509 332.331,268.216 337.858,266.968 343.386,265.769 348.913,264.621 354.441,263.525 359.969,262.484 365.496,261.499 
 371.024,260.573 376.551,259.706 382.079,258.901 387.606,258.158 393.134,257.48 398.661,256.867 404.189,256.32 409.717,255.842 415.244,255.431 420.772,255.09 
 426.299,254.819 431.827,254.618 437.354,254.488 442.882,254.428 448.409,254.44 453.937,254.523 459.465,254.677 464.992,254.901 470.52,255.196 476.047,255.56 
 481.575,255.994 487.102,256.495 492.63,257.064 498.157,257.699 503.685,258.399 509.213,259.162 514.74,259.988 520.268,260.875 525.795,261.821 531.323,262.825 
 536.85,263.885 542.378,264.998 547.906,266.163 553.433,267.379 558.961,268.642 564.488,269.951 570.016,271.303 575.543,272.696 581.071,274.129 586.598,275.597 
 592.126,277.1 
 "/>
 
 
 
 
 1y 
 
 
 
 10y

##    Prob(Zᵉ < 0) for Different Horizons

In [6]:
t = 1:10

ProbNegReturn = fill(NaN,length(t))
for i in t
    ProbNegReturn[i] = ΦNS(0,i*μ,i*σ^2)    #cdf(0) for different horizons
end


plot(t,ProbNegReturn*100,color=:red,linewidth=2,legend=false)
title!("Pr(Ze<0)")
xlabel!("Investment horizon (years)")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 20 
 
 
 30 
 
 
 Pr(Ze<0) 
 
 
 Investment horizon (years) 
 
 
 %

## E(Zᵉ|Zᵉ < 0) for Different Horizons

In [7]:
t = 1:10

CondER = fill(NaN,length(t))
for i in t
    CondER[i] = i*μ - sqrt(i)*σ*ϕNS(-sqrt(i)*μ/σ)/ΦNS(-sqrt(i)*μ/σ)  
end

plot(t,CondER,color=:red,linewidth=2)
title!("E(Ze|Ze<0)")
xlabel!("Investment horizon (years)")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 -30 
 
 
 -20 
 
 
 E(Ze|Ze<0) 
 
 
 Investment horizon (years) 
 
 
 % 
 
 
 
 
 
 
 y1